In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Immigration,AtlasIntel,NaN,12/15/2025,12/19/2025,2315,A,2.050922,2.050922,NaN,41.0,57.0,-16.0,41.17719,55.49537,-14.318180,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,23:01:08 19 Dec 2025
1,Donald Trump (2nd term),Economy,AtlasIntel,NaN,12/15/2025,12/19/2025,2315,A,2.050922,2.050922,NaN,35.0,60.0,-25.0,34.46582,60.05812,-25.592300,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,23:01:05 19 Dec 2025
2,Donald Trump (2nd term),Adults,AtlasIntel,NaN,12/15/2025,12/19/2025,2315,A,2.050922,2.050922,NaN,39.3,59.6,-20.3,38.18242,58.99072,-20.808300,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,23:00:52 19 Dec 2025
3,Donald Trump (2nd term),Trade,AtlasIntel,NaN,12/15/2025,12/19/2025,2315,A,2.050922,2.050922,NaN,35.0,61.0,-26.0,34.20537,61.34979,-27.144420,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,23:01:13 19 Dec 2025
4,Donald Trump (2nd term),All polls,AtlasIntel,NaN,12/15/2025,12/19/2025,2315,A,2.050922,2.050922,NaN,39.3,59.6,-20.3,38.97637,58.36543,-19.389060,https://www.atlasintel.org/poll/usa-national-2...,12/19/2025,23:00:38 19 Dec 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3469,Donald Trump (2nd term),Weak,Ipsos,Reuters,1/20/2025,1/21/2025,1077,A,0.749248,0.000000,NaN,15.0,8.0,7.0,12.83679,6.76224,6.074550,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,23:01:01 19 Dec 2025
3470,Donald Trump (2nd term),Immigration,Ipsos,Reuters,1/20/2025,1/21/2025,1077,A,0.749248,0.000000,NaN,46.0,39.0,7.0,47.89288,39.50993,8.382950,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,23:01:08 19 Dec 2025
3471,Donald Trump (2nd term),Economy,Ipsos,Reuters,1/20/2025,1/21/2025,1077,A,0.749248,0.000000,NaN,42.0,36.0,6.0,44.94718,36.17393,8.773251,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,23:01:05 19 Dec 2025
3472,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800,RV,1.093180,0.000000,NaN,56.0,39.0,17.0,52.47796,42.17751,10.300450,https://insideradvantage.com/first-insideradva...,1/20/2025,23:00:38 19 Dec 2025


In [4]:
# approval[approval['pollster']=='YouGov'][approval['subgroup']=='All polls']

In [5]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'All polls'])]

In [6]:
approval.to_csv('trump_approval_polls_RV.csv')

In [7]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [8]:
approval.shape

(615, 20)

In [9]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [10]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [11]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [12]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [13]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [14]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [15]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [16]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [17]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [18]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(615, 20)

In [19]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,Emerson College,NaN,12/14/2025,12/15/2025,1000,RV,1.375719,1.375719,...,50.3,-9.5,40.87381,52.83479,-11.960980,https://emersoncollegepolling.com/december-202...,12/15/2025,23:00:47 19 Dec 2025,Emerson College,-0.569153
1,Donald Trump (2nd term),Voters,Beacon Research/Shaw & Co. Research,FOX News,12/12/2025,12/15/2025,1001,RV,1.409894,1.329430,...,56.0,-12.0,43.84196,54.89959,-11.057630,https://www.foxnews.com/politics/fox-news-poll...,12/15/2025,23:00:47 19 Dec 2025,Beacon Research/Shaw & Co. Research,-0.635562
2,Donald Trump (2nd term),Voters,Quinnipiac University,NaN,12/11/2025,12/15/2025,1035,RV,1.298924,1.151032,...,54.0,-14.0,42.87872,53.02583,-10.147110,https://poll.qu.edu/poll-release?releaseid=3943,12/15/2025,23:00:47 19 Dec 2025,Quinnipiac University,-0.290484
3,Donald Trump (2nd term),Voters,Echelon Insights,NaN,12/11/2025,12/15/2025,1011,LV,1.250305,1.107405,...,55.0,-10.0,44.48346,54.80188,-10.318420,https://echelonin.wpenginepowered.com/wp-conte...,12/15/2025,23:00:47 19 Dec 2025,Echelon Insights,-0.216672
4,Donald Trump (2nd term),Voters,Marist College,NPR | PBS NewsHour,12/8/2025,12/11/2025,1261,RV,1.430087,1.021664,...,55.0,-15.0,41.15297,54.30745,-13.154480,https://maristpoll.marist.edu/polls/2026-econo...,12/11/2025,23:00:47 19 Dec 2025,Marist College,-0.436610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302,RV,0.795909,0.000000,...,44.0,8.0,51.18822,44.36939,6.818832,https://pro.morningconsult.com/trackers/donald...,1/26/2025,23:00:47 19 Dec 2025,Morning Consult,0.584994
611,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500,LV,0.213379,0.000000,...,43.0,9.0,49.24623,44.64405,4.602180,https://www.rasmussenreports.com/public_conten...,1/24/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015
612,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024,LV,1.211361,0.000000,...,43.0,8.0,50.48346,42.80188,7.681580,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,23:00:47 19 Dec 2025,Echelon Insights,-0.216672
613,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000,RV,0.927446,0.000000,...,39.0,18.0,53.42978,41.67276,11.757020,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,23:00:47 19 Dec 2025,RMG Research,0.278023


In [20]:
approval.shape

(615, 22)

In [21]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     232
YouGov                                                 57
Morning Consult                                        48
RMG Research                                           46
Global Strategy Group/GBAO (Navigator Research)        18
J.L. Partners                                          17
Echelon Insights                                       14
Harris Insights & Analytics                            12
Emerson College                                        12
Quinnipiac University                                  11
American Research Group                                10
John Zogby Strategies                                   9
Clarity Campaign Labs                                   8
Beacon Research/Shaw & Co. Research                     7
Ipsos                                                   7
Marquette University Law School                         6
SSRS                                                    6
McLau

In [22]:
# approval[~approval['tracking'].isna()]

In [23]:
approval['tracking'].value_counts()

tracking
True    232
Name: count, dtype: int64

In [24]:
# full_approval = approval.copy()

In [25]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,Emerson College,NaN,12/14/2025,12/15/2025,1000,RV,1.375719,1.375719,...,50.3,-9.5,40.87381,52.83479,-11.960980,https://emersoncollegepolling.com/december-202...,12/15/2025,23:00:47 19 Dec 2025,Emerson College,-0.569153
1,Donald Trump (2nd term),Voters,Beacon Research/Shaw & Co. Research,FOX News,12/12/2025,12/15/2025,1001,RV,1.409894,1.329430,...,56.0,-12.0,43.84196,54.89959,-11.057630,https://www.foxnews.com/politics/fox-news-poll...,12/15/2025,23:00:47 19 Dec 2025,Beacon Research/Shaw & Co. Research,-0.635562
2,Donald Trump (2nd term),Voters,Quinnipiac University,NaN,12/11/2025,12/15/2025,1035,RV,1.298924,1.151032,...,54.0,-14.0,42.87872,53.02583,-10.147110,https://poll.qu.edu/poll-release?releaseid=3943,12/15/2025,23:00:47 19 Dec 2025,Quinnipiac University,-0.290484
3,Donald Trump (2nd term),Voters,Echelon Insights,NaN,12/11/2025,12/15/2025,1011,LV,1.250305,1.107405,...,55.0,-10.0,44.48346,54.80188,-10.318420,https://echelonin.wpenginepowered.com/wp-conte...,12/15/2025,23:00:47 19 Dec 2025,Echelon Insights,-0.216672
4,Donald Trump (2nd term),Voters,Marist College,NPR | PBS NewsHour,12/8/2025,12/11/2025,1261,RV,1.430087,1.021664,...,55.0,-15.0,41.15297,54.30745,-13.154480,https://maristpoll.marist.edu/polls/2026-econo...,12/11/2025,23:00:47 19 Dec 2025,Marist College,-0.436610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302,RV,0.795909,0.000000,...,44.0,8.0,51.18822,44.36939,6.818832,https://pro.morningconsult.com/trackers/donald...,1/26/2025,23:00:47 19 Dec 2025,Morning Consult,0.584994
611,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500,LV,0.213379,0.000000,...,43.0,9.0,49.24623,44.64405,4.602180,https://www.rasmussenreports.com/public_conten...,1/24/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015
612,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024,LV,1.211361,0.000000,...,43.0,8.0,50.48346,42.80188,7.681580,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,23:00:47 19 Dec 2025,Echelon Insights,-0.216672
613,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000,RV,0.927446,0.000000,...,39.0,18.0,53.42978,41.67276,11.757020,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,23:00:47 19 Dec 2025,RMG Research,0.278023


In [26]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [27]:
# house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+\d.\d\d.*@@([\d\.\+\-]+)')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,-1.999072
1,D,1.017374
2,D,0.632896
3,D,0.791334
4,R,-0.597041
...,...,...
93,R,-2.443137
94,R,-0.446172
95,R,-3.234717
96,R,-4.939213


In [28]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[1]
house_eff

,0,1,house_effect
0,-1,-1.999072,-1.999072
1,1,1.017374,1.017374
2,1,0.632896,0.632896
3,1,0.791334,0.791334
4,-1,-0.597041,-0.597041
...,...,...,...
93,-1,-2.443137,-2.443137
94,-1,-0.446172,-0.446172
95,-1,-3.234717,-3.234717
96,-1,-4.939213,-4.939213


In [29]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-1.999072
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.017374
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.632896
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.791334
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.597041
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.443137
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.446172
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.234717


In [30]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [31]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
7,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/12/2025,12/18/2025,1500,LV,0.852482,0.852482,...,42.24623,54.64405,-12.397820,https://www.rasmussenreports.com/public_conten...,12/18/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,44.361664,53.638336
20,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/5/2025,12/11/2025,1500,LV,0.335032,0.191264,...,44.24623,52.64405,-8.397820,https://www.rasmussenreports.com/public_conten...,12/11/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336
23,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/11/2025,12/17/2025,1500,LV,0.119621,0.112395,...,43.24623,54.64405,-11.397820,https://www.rasmussenreports.com/public_conten...,12/17/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336
24,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/8/2025,12/12/2025,1500,LV,0.159536,0.110858,...,43.24623,53.64405,-10.397820,https://www.rasmussenreports.com/public_conten...,12/12/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,52.638336
25,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/10/2025,12/16/2025,1500,LV,0.117768,0.103515,...,43.24623,54.64405,-11.397820,https://www.rasmussenreports.com/public_conten...,12/16/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Donald Trump (2nd term),Voters,Public Opinion Strategies,Paragon Health Institute,2/25/2025,3/2/2025,1000,RV,1.268962,0.000000,...,50.06376,48.84716,1.216599,https://paragoninstitute.org/medicaid/paragon-...,3/2/2025,23:00:47 19 Dec 2025,Public Opinion Strategies,-0.246974,48.928591,51.071409
379,Donald Trump (2nd term),Voters,RealClear Opinion Research,NaN,4/10/2025,4/12/2025,1000,RV,0.916470,0.000000,...,44.46962,45.59337,-1.123749,https://www.yahoo.com/news/rcp-poll-voters-see...,4/12/2025,23:00:47 19 Dec 2025,NaN,NaN,44.200000,43.500000
231,Donald Trump (2nd term),Voters,American Pulse Research & Polling,NaN,6/23/2025,6/25/2025,633,RV,0.813438,0.000000,...,48.59822,49.63217,-1.033951,https://s3.amazonaws.com/kajabi-storefronts-pr...,6/25/2025,23:00:47 19 Dec 2025,American Pulse Research & Polling,0.800463,49.700000,48.900000
303,Donald Trump (2nd term),Voters,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,5/19/2025,800,RV,1.129986,0.000000,...,46.15481,49.46562,-3.310806,https://pinpointpolicyinstitute.org/wp-content...,5/19/2025,23:00:47 19 Dec 2025,NaN,NaN,46.000000,49.000000


In [32]:
approval.shape

(615, 22)

In [33]:
approval = approval_adj.copy()
approval.shape

(615, 24)

In [34]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'] + approval['sponsors'].fillna('none'))[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
7,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/12/2025,12/18/2025,1500,LV,0.852482,0.852482,...,54.64405,-12.39782,https://www.rasmussenreports.com/public_conten...,12/18/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,44.361664,53.638336,0
20,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/5/2025,12/11/2025,1500,LV,0.335032,0.191264,...,52.64405,-8.39782,https://www.rasmussenreports.com/public_conten...,12/11/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,1
23,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/11/2025,12/17/2025,1500,LV,0.119621,0.112395,...,54.64405,-11.39782,https://www.rasmussenreports.com/public_conten...,12/17/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336,2
24,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/8/2025,12/12/2025,1500,LV,0.159536,0.110858,...,53.64405,-10.39782,https://www.rasmussenreports.com/public_conten...,12/12/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,52.638336,3
25,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/10/2025,12/16/2025,1500,LV,0.117768,0.103515,...,54.64405,-11.39782,https://www.rasmussenreports.com/public_conten...,12/16/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336,4
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/9/2025,12/15/2025,1500,LV,0.116402,0.095208,...,52.64405,-8.39782,https://www.rasmussenreports.com/public_conten...,12/15/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,5
28,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,11/26/2025,12/4/2025,1500,LV,0.438816,0.093918,...,56.64405,-16.39782,https://www.rasmussenreports.com/public_conten...,12/4/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,42.361664,55.638336,6


In [35]:
approval['pollster_id'] = pd.factorize(approval['pollster'] + approval['sponsors'].fillna('none'))[0]

In [36]:
approval['population'] = approval['population'].str.strip(' ')

In [37]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['RV'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['LV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [38]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [39]:
approval['population'].value_counts().index.values

array(['RV', 'LV', 'V'], dtype=object)

In [40]:
approval.shape, pipeline(approval).shape

((615, 26), (611, 26))

In [41]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
7,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/12/2025,2025-12-18,1500,LV,0.852482,0.852482,...,-12.397820,https://www.rasmussenreports.com/public_conten...,12/18/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,44.361664,53.638336,0,0
20,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/5/2025,2025-12-11,1500,LV,0.335032,0.191264,...,-8.397820,https://www.rasmussenreports.com/public_conten...,12/11/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,46.361664,51.638336,1,0
23,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/11/2025,2025-12-17,1500,LV,0.119621,0.112395,...,-11.397820,https://www.rasmussenreports.com/public_conten...,12/17/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336,2,0
24,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/8/2025,2025-12-12,1500,LV,0.159536,0.110858,...,-10.397820,https://www.rasmussenreports.com/public_conten...,12/12/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,52.638336,3,0
25,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,12/10/2025,2025-12-16,1500,LV,0.117768,0.103515,...,-11.397820,https://www.rasmussenreports.com/public_conten...,12/16/2025,23:00:47 19 Dec 2025,Rasmussen Reports,0.087015,45.361664,53.638336,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Donald Trump (2nd term),Voters,Public Opinion Strategies,Paragon Health Institute,2/25/2025,2025-03-02,1000,RV,1.268962,0.000000,...,1.216599,https://paragoninstitute.org/medicaid/paragon-...,3/2/2025,23:00:47 19 Dec 2025,Public Opinion Strategies,-0.246974,48.928591,51.071409,608,69
379,Donald Trump (2nd term),Voters,RealClear Opinion Research,NaN,4/10/2025,2025-04-12,1000,RV,0.916470,0.000000,...,-1.123749,https://www.yahoo.com/news/rcp-poll-voters-see...,4/12/2025,23:00:47 19 Dec 2025,NaN,NaN,44.200000,43.500000,609,70
231,Donald Trump (2nd term),Voters,American Pulse Research & Polling,NaN,6/23/2025,2025-06-25,633,RV,0.813438,0.000000,...,-1.033951,https://s3.amazonaws.com/kajabi-storefronts-pr...,6/25/2025,23:00:47 19 Dec 2025,American Pulse Research & Polling,0.800463,49.700000,48.900000,610,71
303,Donald Trump (2nd term),Voters,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,2025-05-19,800,RV,1.129986,0.000000,...,-3.310806,https://pinpointpolicyinstitute.org/wp-content...,5/19/2025,23:00:47 19 Dec 2025,NaN,NaN,46.000000,49.000000,611,72


In [42]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [43]:
full_approval = approval.copy()

In [44]:
approval = approval[approval['tracking'].isna()]

In [45]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [46]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
6,Donald Trump (2nd term),Voters,YouGov,Economist,12/12/2025,12/15/2025,1453,RV,0.935159,0.878121,...,-10.477410,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/15/2025,23:00:47 19 Dec 2025,YouGov,0.047635,43.249587,54.750413,231,2
17,Donald Trump (2nd term),Voters,YouGov,Economist,12/5/2025,12/8/2025,1380,RV,0.776600,0.390674,...,-9.477409,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/8/2025,23:00:47 19 Dec 2025,YouGov,0.047635,43.249587,53.750413,232,2
21,Donald Trump (2nd term),Voters,YouGov,Economist,11/28/2025,12/1/2025,1456,RV,0.714851,0.156077,...,-12.477410,https://d3nkl3psvxxpe9.cloudfront.net/document...,12/1/2025,23:00:47 19 Dec 2025,YouGov,0.047635,42.249587,55.750413,233,2
35,Donald Trump (2nd term),Voters,YouGov,Economist,11/21/2025,11/24/2025,1511,RV,0.654305,0.048673,...,-11.477410,https://d3nkl3psvxxpe9.cloudfront.net/document...,11/24/2025,23:00:47 19 Dec 2025,YouGov,0.047635,43.249587,55.750413,234,2
39,Donald Trump (2nd term),Voters,YouGov,Yahoo News,11/20/2025,11/24/2025,1132,RV,0.580924,0.038224,...,-8.477409,https://www.scribd.com/document/955164028/Yaho...,11/24/2025,23:00:47 19 Dec 2025,YouGov,0.047635,44.249587,53.750413,235,3


In [47]:
approval[approval['pollster']=='Civiqs']# [['approve', 'disapprove', 'approve_adj', 'disapprove_adj']]

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
298,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,5/17/2025,5/20/2025,1018,RV,1.120966,0.0,...,-4.318291,https://civiqs.com/documents/Civiqs_DailyKos_b...,5/20/2025,23:00:47 19 Dec 2025,Civiqs,0.158562,47.860928,51.139072,573,49
373,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,4/12/2025,4/15/2025,1124,RV,1.149724,0.0,...,-6.318291,https://civiqs.com/reports/2025/04/17/report-a...,4/15/2025,23:00:47 19 Dec 2025,Civiqs,0.158562,46.860928,52.139072,574,49
407,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,2/28/2025,3/3/2025,1031,RV,1.124632,0.0,...,-3.318291,https://civiqs.com/reports/2025/03/04/report-a...,3/3/2025,23:00:47 19 Dec 2025,Civiqs,0.158562,48.860928,51.139072,575,49


In [48]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id', 'pollster_id'], dtype=object)

In [49]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id', 'pollster_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,pollster_id
6,YouGov,Economist,12/12/2025,12/15/2025,1453,RV,43.0,55.0,43.249587,54.750413,-12.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,231,2
17,YouGov,Economist,12/5/2025,12/8/2025,1380,RV,43.0,54.0,43.249587,53.750413,-11.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,232,2
21,YouGov,Economist,11/28/2025,12/1/2025,1456,RV,42.0,56.0,42.249587,55.750413,-14.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,233,2
35,YouGov,Economist,11/21/2025,11/24/2025,1511,RV,43.0,56.0,43.249587,55.750413,-13.000000,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,234,2
39,YouGov,Yahoo News,11/20/2025,11/24/2025,1132,RV,44.0,54.0,44.249587,53.750413,-10.000000,https://www.scribd.com/document/955164028/Yaho...,0.047635,235,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Public Opinion Strategies,Paragon Health Institute,2/25/2025,3/2/2025,1000,RV,51.0,49.0,48.928591,51.071409,2.000000,https://paragoninstitute.org/medicaid/paragon-...,-0.246974,608,69
379,RealClear Opinion Research,NaN,4/10/2025,4/12/2025,1000,RV,44.2,43.5,44.200000,43.500000,0.700001,https://www.yahoo.com/news/rcp-poll-voters-see...,NaN,609,70
231,American Pulse Research & Polling,NaN,6/23/2025,6/25/2025,633,RV,49.7,48.9,49.700000,48.900000,0.799999,https://s3.amazonaws.com/kajabi-storefronts-pr...,0.800463,610,71
303,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,5/19/2025,800,RV,46.0,49.0,46.000000,49.000000,-3.000000,https://pinpointpolicyinstitute.org/wp-content...,NaN,611,72


In [50]:
approval_polls.shape

(372, 15)

In [51]:
def zone_flood_checker(data, pid, date):
    """
    Check whether there are multiple polls from the same pollster + same sponsor (if applicable) within the last two weeks, 
    which could indicate "flooding the zone." Return number of polls from same pollster/sponsor in last two weeks.

    :param data: Polls dataframe.
    :param pid: Pollster ID, specified in the polls dataframe as the column pollster_id.
    :param date: Date at which specified poll is conducted. Corresponds to column end_date in polls dataframe.
    """
    thres = date - pd.Timedelta(weeks=2)
    df = data[ (data['pollster_id'] == pid) & (pd.to_datetime(data['end_date']) >= thres) ]
    return max(df.shape[0], 1)

In [52]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(30))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual): # For pollsters without a quality rating from the Silver Bulletin
            return 0.2
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

   # Multiple polls in short window (aka "flooding the zone")
    def zone_flood_weight(pid, date):
        num_polls = zone_flood_checker(approval_polls, pid, date)
        return 1 / np.sqrt(num_polls)
    state_pivot['zone_flood_weight'] = state_pivot[['pollster_id', 'end_date']].apply(
        lambda x: zone_flood_weight(x['pollster_id'], x['end_date']),
        axis=1
    )

    # Gather the weights together
    state_pivot['total_weights'] = (state_pivot['sample_size_weights'] * state_pivot['time_weights'] * 
                                    state_pivot['quality_weights'] * state_pivot['population_weights'] * state_pivot['zone_flood_weight'])
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [53]:
approval_polls['sample_size'].isna().any()

False

In [54]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
6,YouGov,Economist,12/12/2025,2025-12-15,1453,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,231,2,0.002636,6.812921e-01,0.964021,0.8,0.577350,4.386306e-02
17,YouGov,Economist,12/5/2025,2025-12-08,1380,RV,43.0,54.0,43.249587,53.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,232,2,0.002569,3.981072e-01,0.964021,0.8,0.500000,2.163231e-02
21,YouGov,Economist,11/28/2025,2025-12-01,1456,RV,42.0,56.0,42.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,233,2,0.002639,2.326305e-01,0.964021,0.8,0.447214,1.161330e-02
35,YouGov,Economist,11/21/2025,2025-11-24,1511,RV,43.0,56.0,43.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,234,2,0.002689,1.359356e-01,0.964021,0.8,0.408248,6.310784e-03
39,YouGov,Yahoo News,11/20/2025,2025-11-24,1132,RV,44.0,54.0,44.249587,53.750413,...,https://www.scribd.com/document/955164028/Yaho...,0.047635,235,3,0.002327,1.359356e-01,0.964021,0.8,1.000000,1.337981e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,Public Opinion Strategies,Paragon Health Institute,2/25/2025,2025-03-02,1000,RV,51.0,49.0,48.928591,51.071409,...,https://paragoninstitute.org/medicaid/paragon-...,-0.246974,608,69,0.002187,1.711328e-10,1.209225,0.8,1.000000,1.985855e-11
379,RealClear Opinion Research,NaN,4/10/2025,2025-04-12,1000,RV,44.2,43.5,44.200000,43.500000,...,https://www.yahoo.com/news/rcp-poll-voters-see...,NaN,609,70,0.002187,3.981072e-09,0.200000,0.8,1.000000,7.640767e-11
231,American Pulse Research & Polling,NaN,6/23/2025,2025-06-25,633,RV,49.7,48.9,49.700000,48.900000,...,https://s3.amazonaws.com/kajabi-storefronts-pr...,0.800463,610,71,0.001740,1.165914e-06,0.200000,0.8,1.000000,1.780350e-08
303,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,2025-05-19,800,RV,46.0,49.0,46.000000,49.000000,...,https://pinpointpolicyinstitute.org/wp-content...,NaN,611,72,0.001956,6.812921e-08,0.200000,0.8,1.000000,1.169541e-09


In [55]:
avgs_pipeline(approval_polls, datetime.datetime.today()).shape

(370, 21)

In [56]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [57]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,56.183676,39.238320
1,2025-01-24,53.705713,41.036523
2,2025-01-25,53.705713,41.036523
3,2025-01-26,53.634026,41.178871
4,2025-01-27,50.891752,42.227976
...,...,...,...
327,2025-12-16,43.147650,53.570701
328,2025-12-17,43.147650,53.570701
329,2025-12-18,43.607517,53.262473
330,2025-12-19,43.607517,53.262473


In [58]:
polls_df.shape

(370, 21)

In [59]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
6,YouGov,Economist,12/12/2025,2025-12-15,1453,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,231,2,0.002636,0.681292,0.964021,0.8,0.577350,0.043863
17,YouGov,Economist,12/5/2025,2025-12-08,1380,RV,43.0,54.0,43.249587,53.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,232,2,0.002569,0.398107,0.964021,0.8,0.500000,0.021632
21,YouGov,Economist,11/28/2025,2025-12-01,1456,RV,42.0,56.0,42.249587,55.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,233,2,0.002639,0.232631,0.964021,0.8,0.447214,0.011613


In [60]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [61]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://napolitannews.org/jobapproval...",2025-12-10,2025-12-18,3000 RV,6.646002e-02,50.0,49.0,1.0
1,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-12-12,2025-12-15,1453 RV,4.386306e-02,43.0,55.0,-12.0
2,"<a href=""https://echelonin.wpenginepowered.com...",2025-12-11,2025-12-15,1011 LV,9.707559e-02,45.0,55.0,-10.0
3,"<a href=""https://emersoncollegepolling.com/dec...",2025-12-14,2025-12-15,1000 RV,1.012929e-01,40.8,50.3,-9.5
4,"<a href=""https://pro.morningconsult.com/tracke...",2025-12-12,2025-12-15,2201 RV,1.371719e-02,46.0,52.0,-6.0
...,...,...,...,...,...,...,...,...
365,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,2.135659e-13,52.0,43.0,9.0
366,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,4.952304e-14,52.0,44.0,8.0
367,"<a href=""https://echeloninsights.com/in-the-ne...",2025-01-22,2025-01-24,1024 LV,3.977218e-13,51.0,43.0,8.0
368,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,1.832633e-13,57.0,39.0,18.0


In [62]:
display_df.to_csv('polls_display_table_RV.csv')

In [63]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,0.754850,0.260972
1,2025-01-24,2.646143,1.888459
2,2025-01-25,2.646143,1.888459
3,2025-01-26,2.587478,1.925188
4,2025-01-27,3.826243,2.060159
...,...,...,...
327,2025-12-16,2.393664,2.296343
328,2025-12-17,2.393664,2.296343
329,2025-12-18,2.884335,2.501446
330,2025-12-19,2.884335,2.501446


In [64]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,56.183676,39.238320,16.945356
1,2025-01-24,53.705713,41.036523,12.669190
2,2025-01-25,53.705713,41.036523,12.669190
3,2025-01-26,53.634026,41.178871,12.455155
4,2025-01-27,50.891752,42.227976,8.663776
...,...,...,...,...
327,2025-12-16,43.147650,53.570701,-10.423051
328,2025-12-17,43.147650,53.570701,-10.423051
329,2025-12-18,43.607517,53.262473,-9.654955
330,2025-12-19,43.607517,53.262473,-9.654955


In [65]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,56.183676,39.238320,57.663182,54.704171,39.749825,38.726816
1,2025-01-24,53.705713,41.036523,58.892154,48.519273,44.737904,37.335143
2,2025-01-25,53.705713,41.036523,58.892154,48.519273,44.737904,37.335143
3,2025-01-26,53.634026,41.178871,58.705483,48.562569,44.952239,37.405503
4,2025-01-27,50.891752,42.227976,58.391188,43.392316,46.265888,38.190064
...,...,...,...,...,...,...,...
327,2025-12-16,43.147650,53.570701,47.839231,38.456070,58.071534,49.069869
328,2025-12-17,43.147650,53.570701,47.839231,38.456070,58.071534,49.069869
329,2025-12-18,43.607517,53.262473,49.260813,37.954222,58.165308,48.359638
330,2025-12-19,43.607517,53.262473,49.260813,37.954222,58.165308,48.359638


In [66]:
# Export approval ratings
approval_trendline.to_csv('approval_stats_RV.csv')

In [67]:
polls_df.shape

(370, 22)

In [68]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [69]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [70]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [71]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [72]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,16.945356,1.015822
1,2025-01-24,12.669190,4.528228
2,2025-01-25,12.669190,4.528228
3,2025-01-26,12.455155,4.485815
4,2025-01-27,8.663776,5.500321
...,...,...,...
327,2025-12-16,-10.423051,3.588698
328,2025-12-17,-10.423051,3.588698
329,2025-12-18,-9.654955,4.506655
330,2025-12-19,-9.654955,4.506655


In [73]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,16.945356,1.015822,18.936366,14.954345
1,2025-01-24,12.669190,4.528228,21.544516,3.793864
2,2025-01-25,12.669190,4.528228,21.544516,3.793864
3,2025-01-26,12.455155,4.485815,21.247353,3.662957
4,2025-01-27,8.663776,5.500321,19.444405,-2.116852
...,...,...,...,...,...
327,2025-12-16,-10.423051,3.588698,-3.389203,-17.456899
328,2025-12-17,-10.423051,3.588698,-3.389203,-17.456899
329,2025-12-18,-9.654955,4.506655,-0.821911,-18.487999
330,2025-12-19,-9.654955,4.506655,-0.821911,-18.487999


In [74]:
net_approval.to_csv('net_approval_RV.csv')

### Auxiliary Graph: Individual Pollsters

In [76]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id',
       'pollster_id'], dtype=object)

In [77]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
appr = appr[~appr['pred_plus_minus'].isna()]
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'Global Strategy Group/GBAO (Navigator Research)', 'J.L. Partners',
       'Echelon Insights', 'Emerson College',
       'Harris Insights & Analytics', 'Quinnipiac University',
       'American Research Group', 'John Zogby Strategies',
       'Clarity Campaign Labs', 'Ipsos',
       'Beacon Research/Shaw & Co. Research', 'Marist College',
       'InsiderAdvantage', 'SSRS', 'Marquette University Law School',
       'McLaughlin & Associates', 'TIPP Insights', 'Cygnal', 'Wick',
       'Civiqs', 'Fabrizio Lee & Associates/Impact Research',
       'Targoz Market Research', 'co/efficient', 'High Point University',
       'The Winston Group', 'The New York Times/Siena College',
       'Trafalgar Group', 'GBAO', 'NewsNation',
       'Hart Research Associates/Public Opinion Strategies',
       'Blueprint Polling', 'OH Predictive Insights',
       'North Star Opinion Research', 'OnMessage Inc.',
       'Normington, P

In [78]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [79]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [80]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [81]:
appr.shape

(584, 27)

In [82]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [83]:
get_ppm('YouGov')

0.047635015

In [84]:
indiv_pollster_avg('YouGov')

(43.019654796569135,
 54.86674308474112,
 0.48933798407434487,
 0.7193981537784713,
 -11.847088288171967,
 1.1086086955978987)

In [85]:
indiv_pollster_avg('RMG Research')

(48.19805231418898,
 50.33333633998012,
 1.9464040500981878,
 1.5442252094350777,
 -2.1352840257911616,
 3.472591842656574)

In [86]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [87]:
# nets

In [88]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
10,American Research Group,-26.723043,1.045747,0.818141,10
17,SSRS,-21.998431,0.125238,0.966835,6
27,High Point University,-18.999130,0.088491,0.411680,2
13,Ipsos,-16.610781,2.006344,0.442898,7
15,Marist College,-15.203043,0.640681,-0.436610,6
4,Global Strategy Group/GBAO (Navigator Research),-14.202817,0.984215,0.804988,18
9,Quinnipiac University,-14.002256,0.072813,-0.290484,11
32,NewsNation,-13.000000,0.000656,0.614459,2
14,Beacon Research/Shaw & Co. Research,-12.380126,1.329506,-0.635562,7
33,Hart Research Associates/Public Opinion Strate...,-12.000000,0.000950,-0.250934,2


In [89]:
# indiv_avgs.to_csv('indiv_pollster_avgs_RV.csv')

In [90]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-2.79174004]), -7.548962607250128)

In [91]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.21955093])